# 8_Tables.ipynb
This notebook will create all of the tables needed for the publication.

In [1]:
import hydrofunctions as hf

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
#%matplotlib --list
#%matplotlib qt
#%matplotlib ipympl
%matplotlib inline

import analysis_functions as my

from scipy.stats import linregress

my.print_versions()

Package Versions
----------------
Python: 3.10.16
numpy: 1.26.4
pandas: 2.2.3
scipy: 1.15.1
statsmodels: 0.14.4
pangoin: not imported
matplotlib: 3.10.0
seaborn: 0.13.2
hydrofunctions: 0.2.4
analysis_funtions: 2025.06.27


In [2]:
"""
from IPython.display import display, HTML

with pd.option_context('display.max_colwidth', 220):
  display(HTML(d1.to_html()))

# Export dataframe to an excel file
df.to_excel("file_name.xlsx")

# Creating lots of sheets all at once
with pd.ExcelWriter('Tables.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Table_1')
    df2.to_excel(writer, sheet_name='Table_2')

    # Appending sheets one at a time
    with pd.ExcelWriter('Tables.xlsx', mode='a') as writer:  
        dataframe.to_excel(writer, sheet_name=sheet_name)
"""

class TableSaver(object):
    def __init__(self, filename):
        self.filename = filename
        self.brand_new = True
        print(f"Initializing {self.filename}. Brand new: {self.brand_new}")
    def __call__(self, sheet_name, dataframe):
        if self.brand_new:
            self.brand_new = False
            with pd.ExcelWriter(self.filename) as writer:  
                dataframe.to_excel(writer, sheet_name=sheet_name)
        else:
            with pd.ExcelWriter(self.filename, mode='a', if_sheet_exists='replace') as writer:  
                dataframe.to_excel(writer, sheet_name=sheet_name)
            
table = "tables.xlsx"
export_table = TableSaver("Tables.xlsx")
#use like this: export_table(sheet_name, dataframe)



def func(_static={'counter': 0}):
     _static['counter'] += 1
     print(_static['counter'])

def write_excel(table_name, sheet_name, dataframe):
    # Appending sheets one at a time
    with pd.ExcelWriter(table_name, mode='a') as writer:  
        dataframe.to_excel(writer, sheet_name=sheet_name)

Initializing Tables.xlsx. Brand new: True


# Table 1: stations

USGS ID#	Drainage km2	Median Q m3s-1 	Gauge Datum NGDV 1929   ft
m	‘Cease to flow’ stage (m)	Missing data
(% of 140,352)
(Bower)	01541000	816	---	1207.14
xx	xx	
(Curwensville Lake)	01541180	945	---	0	--	
Curwensville	01541200	951	10.0	1124.66
342.80 m	0.58 m	491 (0.3%)
Hyde	01541303	1,228	14.2	1093.90
333.42 m	0.65 m	14812 (10.6%)
Karthaus	01542500	3,787	40.2	830.59
253.16 m	-0.02 m	4901 (3.5%)
Renovo	01545500	7,705	84.7	634.19
193.30 m	-0.47 m	11858 (8.4%)
(Lock Haven)	01545800	8,664	---	534.519		
(Jersey Shore)	01549760	13,533	---	513.19		
Williamsport	01551500	14,716	162.0	494.98
150.87 m	- 0.58 m	12430 (8.9%)
Table 1. The stream gauges used in this study, plus five nearby stream gauges (in parentheses) that were not used in the study.

USGS ID#	Drainage km2	Median Q m3s-1 	Gauge Datum NGDV 1929   ft
m	‘Cease to flow’ stage (m)	Missing data
(% of 140,352)



In [3]:
# Table 1: Station site data

site_index = ['01541000', '01541180', '01541200', '01541303', '01542500', '01545500', '01545800', '01549760', '01551500', '01553025', '01553240', '01553500']
site_data = {'name': ['(Bower)', '(Curwensville Lake)', 'Curwensville', 'Hyde', 'Karthaus', 'Renovo', '(Lock Haven)', '(Jersey Shore)', 'Williamsport', 'Allenwood',
                     'Milton', 'Lewisburg'],
             'gauge_datum':[1207.14, 0, 1124.66, 1093.90, 830.59, 634.19, 534.519, 513.19, 494.98, None, None, None],
             'Drainage km²':[816, 945, 951, 1228, 3787, 7705, 8664, 13533, 14716, None, None, None]
            }
txt_stations = pd.DataFrame(data=site_data, index=site_index)
txt_stations['gauge_datum_m'] = txt_stations['gauge_datum'] * 0.3048
# The "Susquehanna_Merge_Table.txt" file was created in ArcGIS from NHD; it is a table of the
# river segments from the NHD merged into reaches that stretch between each stream gauge on 
# the river. Length is given in meters. Stream gauges are listed by their USGS ID#.
reaches = pd.read_csv('Susquehanna_Merge_Table.txt', dtype={'US_Gauge':str, 'DS_Gauge':str})
reaches.sort_values(by=['US_Gauge'], ascending=False, inplace=True)
# The 'positionNHD' field is the distance from the gauge to the Chesapeake Bay in meters.
reaches.loc[:,'dist_km'] = (reaches.loc[:,'Shape_Length'].cumsum()/1000).round(1)
posit_stations = reaches.loc[reaches['US_Gauge'].isin(txt_stations.index.values)].set_index('US_Gauge').loc[:,['dist_km']]
cease_stations = pd.read_parquet('stations.parquet').loc[:,['StageQ0root']].rename(columns={'StageQ0root':"'Cease-to-flow' stage (m)"})  #Just pull 'cease-to-flow' from the stations table
# Calculate median discharge as the median of all values in the dataset created in LTA_0_Data-Download. units = cubic meters / second
Q = pd.read_parquet('WBdata-discharge.parquet')
stage = pd.read_parquet('WBdata-stage.parquet')
median = Q.median()
median.name = 'Median Q m³s^-1'
def missing(series, prefix):
    missing_n = len(series) - series.count()
    missing_pct = 100 * missing_n / len(series)
    return pd.DataFrame([missing_n, missing_pct], index=[f'{prefix} count', f'{prefix} %']).T

allstations = pd.concat([txt_stations, median, cease_stations, missing(Q, 'discharge'), missing(stage, 'stage')], axis=1)
out_table1 = allstations.reset_index(names='USGS ID#').set_index('name').iloc[:-3]

#write_excel(table, 'Table 1', out_table1)
export_table('Table 1', out_table1)
out_table1

,USGS ID#,gauge_datum,Drainage km²,gauge_datum_m,Median Q m³s^-1,'Cease-to-flow' stage (m),discharge count,discharge %,stage count,stage %
name,,,,,,,,,,
(Bower),01541000,1207.140,816.0,367.936272,NaN,NaN,NaN,NaN,NaN,NaN
(Curwensville Lake),01541180,0.000,945.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
Curwensville,01541200,1124.660,951.0,342.796368,9.967514,0.579685,491.0,0.349835,265.0,0.188811
Hyde,01541303,1093.900,1228.0,333.420720,14.186717,0.645428,14812.0,10.553466,41.0,0.029212
Karthaus,01542500,830.590,3787.0,253.163832,40.209856,-0.016348,4901.0,3.491935,5.0,0.003562
Renovo,01545500,634.190,7705.0,193.301112,84.667232,-0.474518,11858.0,8.448757,56.0,0.039900
(Lock Haven),01545800,534.519,8664.0,162.921391,NaN,NaN,NaN,NaN,NaN,NaN
(Jersey Shore),01549760,513.190,13533.0,156.420312,NaN,NaN,NaN,NaN,NaN,NaN
Williamsport,01551500,494.980,14716.0,150.869904,161.972096,-0.581263,12430.0,8.856304,50.0,0.035625


# Table 2: Reaches

Length (m)	Head (m)	gradient	Qus/Qds	Aus/Ads

In [4]:
new_cols = {'length':'Length (m)', 'head':'Head (m)', 'deltaQ':'Qus/Qds', 'deltaA':'Aus/Ads'}
out_table2 = pd.read_parquet('reaches.parquet').rename(columns=new_cols).iloc[:,2:7]

#write_excel(table, 'Table 2', out_table2)
export_table('Table 2', out_table2)
out_table2

,Length (m),Head (m),gradient,Qus/Qds,Aus/Ads
Curwensville-Hyde,13034,9.309905,0.000714,1.423295,1.291272
Hyde-Karthaus,71700,80.918664,0.001129,2.834331,3.083876
Karthaus-Renovo,55524,60.320890,0.001086,2.105634,2.034592
Renovo-Williamsport,93772,42.537953,0.000454,1.913043,1.909929


# Table 3: length and number of segments

Level	# of segments	# of observations per segment	Length of time per segment
0	1	140,352	4 years (1,462 days)
1 	2	70,176	2 years (731 days)
2	4	35,088	1 year (365 days, 12 hrs)
3	8	17,544	6 months (182 days, 18 hrs)
4	16	8,772	3 months (91 days, 9 hrs)
5	32	4,386	45 days, 16.5 hrs
6	64	2,193	22 days, 20 hrs, 15 min
7	128	1,097	11 days, 10 hrs
Table 3. The number and length of segments for each level.

In [5]:
# Table 3: length and number of segments
row = []
for level in range(0,8):
    data = {}
    data['Level'] = level
    segments = 2**level
    data['# of segments'] = segments
    n_obs = 140352 / segments
    data['# of observations per segment'] = n_obs
    data['Length of time per segment'] = pd.Timedelta('15 minutes') * n_obs
    row.append(data)
out_table3 = pd.DataFrame(row)

#write_excel(table, 'Table 3', out_table3)
export_table('Table 3', out_table3)
out_table3

,Level,# of segments,# of observations per segment,Length of time per segment
0,0,1,140352.0,1462 days 00:00:00
1,1,2,70176.0,731 days 00:00:00
2,2,4,35088.0,365 days 12:00:00
3,3,8,17544.0,182 days 18:00:00
4,4,16,8772.0,91 days 09:00:00
5,5,32,4386.0,45 days 16:30:00
6,6,64,2193.0,22 days 20:15:00
7,7,128,1096.5,11 days 10:07:30


# Table 4: Comparison statistics for each method and the aggregated validation data.

Method	Plausible Level 4	Max Level	Robust Slope Level 4	Robust MAE Level 4	Count Level 4	Robust Slope Level 7	Robust MAE Level 7	Count Level 7



In [6]:
# Table 4: Comparison statistics for each method and the aggregated validation data.
# From 5a_Execute-analysis.ipynb

import statsmodels.api as sm
import scipy.stats as stats



output = pd.read_parquet('xcorr_output.parquet')

first_table = pd.DataFrame()

plausible_mask = output['plausible_c']
no_mask = pd.Series(True, index=output.index)
# What % of the observations are within the plausible range?
first_table['4 %plaus'] = plausible_mask.loc[(slice(None),4)].groupby('method', sort=False).mean()
first_table['7 %plaus'] = plausible_mask.loc[(slice(None),7)].groupby('method', sort=False).mean()

# Highest Level under 10% plausible
plaus_level = plausible_mask.groupby(['method','level']).mean()
level_90_mask = plaus_level.gt(0.9)
max_plaus_level = plaus_level.loc[level_90_mask].groupby('method').idxmin()
first_table['max level'] = pd.DataFrame(max_plaus_level.tolist(), columns=['method', 'max_level'], index=max_plaus_level.index)['max_level']

output_plaus_4 = output.loc[plausible_mask].loc[(slice(None),4),:]
c_result_vals = output_plaus_4.groupby(['method'])['celerity'].agg(['mean', 'median', stats.iqr , 'count'])
c_result_vals = c_result_vals.rename(columns=lambda x: f"4 {x}(plaus)")
first_table = first_table.join(c_result_vals)

def regress_vc(group, level, plaus_filter=False):
    #print(group.name)  

    if plaus_filter:
        mask = group['plausible_c']
        mask_label = 'plaus'
    else:
        mask = pd.Series(True, index=group.index)
        mask_label = 'all'
    clean_group = group.loc[mask].loc[(slice(None), level),['avg_val_c','celerity']].dropna()
    y = clean_group['avg_val_c']
    x = clean_group['celerity']
    
    if len(x) < 4:
        robust_mae = 0.0
        robust_slope = 1.0
        #robust_intercept = 0.0
        linear_mae = 0.0
        linear_slope = 1.0
        #linear_intercept = 0.0
    elif group.name == 'validation': 
        # Because the aggregated validation data will have a perfect fit with itself, we
        # need to avoid doing the calculation, which will result in dividing by zero, etc.
        robust_mae = 0.0
        robust_slope = 1.0
        #robust_intercept = 0.0
        linear_mae = 0.0
        linear_slope = 1.0
        #linear_intercept = 0.0
    else:
        #x2 = sm.add_constant(x)
        robust = sm.RLM(y, x).fit()
        robust_mae = np.mean(np.abs(robust.resid)) 
        robust_slope = robust.params.iloc[0]
        #robust_intercept = robust.params.iloc[0]
        
        linear = linregress(x,y)
        linear_fit = (linear.slope * x) + linear.intercept
        resid = y - linear_fit
        linear_mae = np.mean(np.abs(resid))
        linear_slope = linear.slope
        linear_intercept = linear.intercept
    
    #print(f"{group.name} {level} {mask_label}")
    #print(f"        linear:  y = {linear_slope} * x + {linear_intercept}  MAE:{linear_mae}")
    #print(f"        robust:  y = {robust_slope} * x + {robust_intercept}   MAE:{robust_mae}\n")
    #print(robust.params)

    col_names = [f'{level} robust slope({mask_label})',  f'{level} robust mae({mask_label})', f'{level} linear slope({mask_label})', f'{level} linear mae({mask_label})', f'{level} count({mask_label})']

    return pd.Series([robust_slope, robust_mae, linear_slope, linear_mae, len(x)], index=col_names)

grouped = output.groupby('method', sort=False)
regression4plaus = grouped.apply(regress_vc, 4, plaus_filter=True)
regression4all = grouped.apply(regress_vc, 4, plaus_filter=False)
#regression5 = grouped.apply(regress_vc, 5)
#regression6 = grouped.apply(regress_vc, 6)
regression7plaus = grouped.apply(regress_vc, 7, plaus_filter=True)
regression7all = grouped.apply(regress_vc, 7, plaus_filter=False)


quick_table = pd.concat([first_table,regression4all,regression4plaus,regression7all,regression7plaus], axis=1)

all_cols = ['4 %plaus', '7 %plaus', 'max level', '4 mean(plaus)', '4 median(plaus)',
       '4 iqr(plaus)', '4 count(plaus)', '4 robust slope(all)',
       '4 robust mae(all)', '4 linear slope(all)', '4 linear mae(all)',
       '4 count(all)', '4 robust slope(plaus)', '4 robust mae(plaus)',
       '4 linear slope(plaus)', '4 linear mae(plaus)', '4 count(plaus)',
       '7 robust slope(all)', '7 robust mae(all)', '7 linear slope(all)',
       '7 linear mae(all)', '7 count(all)', '7 robust slope(plaus)',
       '7 robust mae(plaus)', '7 linear slope(plaus)', '7 linear mae(plaus)',
       '7 count(plaus)']
#Select the columns we want by index# and their order
cols = [all_cols[i] for i in [2, 0, 3, 6, 11, 8, 7, 1, 16, 13, 12]]
final_cols = [
    'max level', 
    '4 %plaus',
    '4 count(plaus)',
    '4 mean(plaus)', 
    #'4 median(plaus)',
    #'4 iqr(plaus)', 
    #'4 robust slope(all)',
    #'4 robust mae(all)', 
    #'4 linear slope(all)', 
    #'4 linear mae(all)',
    #'4 count(all)', 
    #'4 robust slope(plaus)',
    #'4 robust slope(all)',
    '4 robust mae(plaus)',
    #'4 linear slope(plaus)', 
    #'4 linear mae(plaus)', 
    #'4 count(plaus)',
    '7 %plaus',
    '7 count(plaus)',
    #'7 robust slope(all)', 
    #'7 robust mae(all)', 
    #'7 linear slope(all)',
    #'7 linear mae(all)', 
    #'7 count(all)', 
    #'7 robust slope(plaus)',
    #'7 robust slope(all)',
    '7 robust mae(plaus)', 
    #'7 linear slope(plaus)', 
    #'7 linear mae(plaus)',
]


formatters = {'4 %plaus': '{:,.1%}'.format, 
              '7 %plaus': '{:,.1%}'.format, 
              '4 count(plaus)':'{:.0f}'.format,
              '4 count(all)':'{:.0f}'.format,
              '7 count(plaus)':'{:.0f}'.format,
              '4 mean(plaus)':'{:.2f}'.format, 
              '4 median(plaus)':'{:.2f}'.format,
              '4 robust slope(plaus)':'{:.4f}'.format,
              '7 robust slope(plaus)':'{:.4f}'.format,
              '4 robust mae(plaus)':'{:.3f}'.format,
              '7 robust mae(plaus)':'{:.3f}'.format,
             }


final_quick_table = (quick_table.loc[:, final_cols]
    .sort_values(['4 %plaus', '7 %plaus', 'max level'], ascending=False)
    .rename(index={'validation':'agg validation'})
    .style.format(formatters)
    .set_sticky(axis="index")
)
#print(formatted_table.to_html())  #Easier to cut and paste this way

#write_excel(table, 'Table 4', final_quick_table)
export_table('Table 4', final_quick_table)

final_quick_table#.style.set_sticky(axis="index")

,max level,4 %plaus,4 count(plaus),4 count(plaus),4 mean(plaus),4 robust mae(plaus),7 %plaus,7 count(plaus),7 robust mae(plaus)
method,,,,,,,,,
agg validation,5,100.0%,64,64,1.50,0.000,49.8%,255,0.000
band1d130m-900,7,96.9%,62,62,1.43,0.391,92.6%,248,0.362
band1d130m-dt-900,7,96.9%,62,62,1.43,0.391,92.6%,248,0.362
band7d130m,5,93.8%,60,60,2.09,0.392,84.0%,232,0.429
band7d130m-900,5,93.8%,60,60,2.09,0.392,84.0%,232,0.429
band3d130m-900,4,92.2%,59,59,1.76,0.357,93.8%,244,0.348
smooth130diff-900,6,92.2%,59,59,1.60,0.317,92.0%,247,0.381
smooth1daydiff,4,90.6%,58,58,1.90,0.331,87.7%,238,0.436
band7d1d,4,90.6%,58,58,2.40,0.407,79.1%,216,0.577


# Table 5. Distribution statistics for wave celerity by reach and source. 

reach	method	count	mean	std	   min	  25%	50%	 75%	max

1-day, 130-minute band-pass filtered depth data split into ~23-day series (level 6), after removing implausible values

In [8]:
xcorr = pd.read_parquet('xcorr_output.parquet')
plaus_mask = xcorr['plausible_c']
xcorr_select = xcorr.loc[plaus_mask].loc[('band1d130m-900', 6),'celerity']
xcorr_desc = xcorr_select.groupby('reach').describe()
validation_reaches = pd.read_parquet('validation_reaches.parquet')
val_desc = validation_reaches.groupby('reach')['c'].describe()
test = pd.concat([val_desc, xcorr_desc], keys=['Validation', 'Cross-Correlation'], names=['source', 'reach'])#.swaplevel().sort_level([0], sort_remaining=False)
#test.groupby(['reach', 'source']).apply(lambda x: x)#.sort_level([0], sort_remaining=False)
out_table5 = test.swaplevel().sort_values('reach')

#write_excel(table, 'Table 5', out_table5)
export_table('Table 5', out_table5)
out_table5

C:\Users\marti\AppData\Local\Temp\ipykernel_20144\2670804052.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  xcorr_select = xcorr.loc[plaus_mask].loc[('band1d130m-900', 6),'celerity']


count      mean       std       min  \
reach               source                                                   
Curwensville-Hyde   Validation          93.0  0.896955  0.375848  0.212974   
                    Cross-Correlation   63.0  0.991171  0.272498  0.517222   
Hyde-Karthaus       Validation          83.0  1.625949  0.650232  0.766026   
                    Cross-Correlation   60.0  1.413329  0.589573  0.118025   
Karthaus-Renovo     Validation          94.0  1.836387  0.789851  0.717364   
                    Cross-Correlation   61.0  1.630779  0.966442  0.105459   
Renovo-Williamsport Validation          81.0  1.587171  0.611925  0.609305   
                    Cross-Correlation   57.0  1.243845  0.697087  0.125683   

                                            25%       50%       75%       max  
reach               source                                                     
Curwensville-Hyde   Validation         0.629662  0.851895  1.034444  2.068889  
                    Cross-Correlation  0.743167  1.034444  1.206852  1.609136  
Hyde-Karthaus       Validation         1.274748  1.503145  1.810606  4.425926  
                    Cross-Correlation  1.011910  1.448485  1.731884  3.319444  
Karthaus-Renovo     Validation         1.370963  1.645448  2.109628  5.608485  
                    Cross-Correlation  1.028222  1.504715  1.927917  5.608485  
Renovo-Williamsport Validation         1.255315  1.510016  1.827914  4.341296  
                    Cross-Correlation  0.766111  1.302389  1.680502  2.976889